In [1]:
import pandas as pd
import random
import numpy as np
import networkx as nx
import nltk
import tensorflow as tf
import collections
import math
import scipy.spatial as ssp

In [5]:
def load():
    ppp = nltk.data.load('../../Downloads/ppp.txt', encoding='utf8')
    words_p = nltk.tokenize.wordpunct_tokenize(ppp)[130:]
    alw = nltk.data.load('../../Downloads/alw.txt', encoding='utf8')
    words_a = nltk.tokenize.wordpunct_tokenize(alw)[143:]
    words = words_a + words_p
    return get_sentences(words)


In [7]:
def get_sentences(words):
    sentences_word  = []
    sent_word = []
    for index, word in enumerate(words):
        if word in ['?','.','!']:
            sent_word += [word]
            sentences_word += [sent_word]
            sent_word = []
        else:
            sent_word += [word]
    return sentences_word

In [146]:
def in_embed_form(batch, KB, dic):
    # at some point instead of just randomly choosing,
    # we can use a neato estimation function for it.
    NK_pre = np.array(list(map(lambda word: 
                     KB[dic[word]] if word in dic \
                      else np.random.uniform(size=(1, 100)), batch)))
    NK = np.concatenate([NK_pre[:-1], NK_pre[1:]], axis=1)
    return NK

In [203]:
def find_X_sets(NK, KB, alpha):
    # NK is new knowledge
    # KB is our knowledge base so far
    # alpha is the deviation from the mean distance a good vector
    # to model our ideal will be
    x_ideal = NK[:, 1] - NK[:, 0].reshape(len(NK), NK.shape[-1])
    dists = vector_cos5(KB, x_ideal[0])
    return dists
    X = np.where((dists > 0) & (dists < alpha)) 
    return X

In [229]:
sentences = load()

In [217]:
NK = in_embed_form(["hello", "good-bye", "woops"], [], [])

KB = np.random.random_sample(size=(500, 100))

In [148]:
NK.shape

(2, 2, 100)

In [174]:
shf = np.arange(len(KB))
np.random.shuffle(shf)
s_dists = ((KB - KB[shf])**2).sum(axis=1)
alpha = (s_dists.mean(axis=0) + s_dists.std(axis=0))

In [249]:
import operator
def dot_product2(v1, v2):
    return sum(map(operator.mul, v1, v2), axis=0)


def vector_cos5(v1, v2):
    prod = dot_product2(v1, v2)
    len1 = np.sqrt(dot_product2(v1, v1))
    len2 = np.sqrt(dot_product2(v2, v2))
    return prod / (len1 * len2)


In [178]:
s_dists.std(axis=0)

2.0807058835338248

In [204]:
X = find_X_sets(NK, KB, alpha)

In [219]:
test = KB[np.random.randint(0, len(KB), size=(100))]

In [250]:
mhm = np.stack((pd.DataFrame(KB).apply(
            lambda row: vector_cos5(row, test_row).argmin(axis=0), axis=1)
          for test_row in test), axis=0)

TypeError: ('sum() takes no keyword arguments', 'occurred at index 0')

In [247]:
mhm

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Let $KB$ be the set of all recorded vectors, then
$$ X = \big( x \in KB \text{ s.t. } \mid dist(a,b) - x \mid \, \le \alpha \big)$$ where $\alpha$ represents the density our graph, $\epsilon$ is a noise term, and 

$$ X_S \sim N(X, \eta S + \epsilon)$$ where $S = p(S, x,y)$, then

$$ p(x\mid a,b) = 
\frac{dist^2(a,x^\ast_0) 
+ dist^2(b, x^\ast_1)}{\sum_{x\in {X_S}}dist^2(a,x_0) 
+ dist^2(b, x_1)} $$



If $ X_S $ is empty, then add $x = dist(a,b)$ to KB